# 4a – Week 4: Setting up your own database 

In [ ]:
#Have a look at the table before and after the insertion to make sure everything went fine. The automarker will run a SELECT * FROM Star query to check the result. 
INSERT INTO Star (kepler_id, t_eff, radius) VALUES
  (7115384, 3789,27.384),
  (8106973, 5810, 0.811),
  (9391817, 6200, 0.958);
  
 SELECT * FROM Star 

## 
Your task is to fix a Planet table has been corrupted.

There are two problems: some of the rows were copied with a negative radius and the unconfirmed planets were given fake Kepler names. To fix this, you'll have to:

    Update the Kepler names of planets which don't have a confirmed status (replace the kepler_name with NULL);
    Delete rows where the radius is negative.

Have a look at the table to find out what needs to be changed.


In [ ]:
UPDATE Planet
 SET kepler_name = NULL
 WHERE UPPER(status) != 'CONFIRMED';

DELETE FROM Planet
 WHERE radius < 0;
 
 SELECT * FROM Planet 

### ow you've ready to create your first table. Your task is to set up a new Planet table and fill it with the planets listed below.

Your table should consist of the following attributes in this order:

    kepler_id, as type INTEGER
    koi_name, as type VARCHAR(15)
    kepler_name, as type VARCHAR(15)
    status, as type VARCHAR(20)
    radius, as type FLOAT

Add the following constraints:

    make all attributes NOT NULL except kepler_name;
    make the koi_name a unique attribute.

Finally, insert the following planets into your new table: 

In [ ]:
CREATE TABLE Planet  (
  kepler_id INTEGER NOT NULL ,
  koi_name VARCHAR(15) NOT NULL UNIQUE ,
  kepler_name VARCHAR(15),
  status VARCHAR(20)NOT NULL,
  radius FLOAT NOT NULL
);

INSERT INTO Planet 
(kepler_id, koi_name, kepler_name, status, radius)
 VALUES(6862328, 'K00865.01', NULL, 'CANDIDATE', 119.021);
 
INSERT INTO Planet 
 (kepler_id, koi_name, kepler_name, status, radius)
 VALUES(10187017, 'K00082.05', 'Kepler-102 b', 'CONFIRMED', 5.286);
 
INSERT INTO Planet 
 (kepler_id, koi_name, kepler_name, status, radius)
 VALUES(10187017, 'K00082.04', 'Kepler-102 c', 'CONFIRMED', 7.071);

### You can now create the Star and Planet tables that we've been querying throughout these activities and fill the tables using the two CSV files which contain the star and planet data.

The tables and CSV files should contain the columns as follows: 

In [ ]:
CREATE TABLE Star (
  kepler_id INTEGER PRIMARY KEY ,
  t_eff INTEGER NOT NULL,
  radius FLOAT NOT NULL 
);
  
CREATE TABLE Planet (
  kepler_id INTEGER REFERENCES Star (Kepler_ID),
  koi_name VARCHAR (20) PRIMARY KEY,
  kepler_name  VARCHAR (20),
  status   CHAR (20) NOT NULL,
  period   FLOAT,
  radius  FLOAT,
  t_eq INTEGER
 
);
COPY Star (kepler_id, t_eff, radius) FROM 'stars.csv' CSV;
COPY Planet (kepler_id, koi_name, kepler_name, status, period, radius, t_eq) FROM 'planets.csv' CSV;
  

In [ ]:
ALTER TABLE Star
  ADD COLUMN ra FLOAT,
  ADD COLUMN decl FLOAT;
DELETE FROM Star;

COPY Star (kepler_id, t_eff, radius, ra, decl) FROM 'stars_full.csv' CSV;

#### 4b – Week 4: Combining SQL and Python 

In [ ]:
import psycopg2

def select_all(table):

  conn = psycopg2.connect('dbname=db user=grok')
  cursor = conn.cursor()

  query = 'SELECT * FROM ' + table + ';'
  
  cursor.execute(query)
  records = cursor.fetchall()

  return records


In [ ]:
import numpy as np
import psycopg2

def column_stats(table, col):
  conn = psycopg2.connect(dbname='db', user='grok')
  cursor = conn.cursor()

  query = 'SELECT ' + col + ' FROM ' + table + ';'
  cursor.execute(query)
  column = np.array(cursor.fetchall())
  return np.mean(column), np.median(column)

In [ ]:
# Write your query function here

import numpy as np

def query(f_name):
  data = np.loadtxt(f_name, delimiter=',', usecols=(0, 2))
  return data[data[:, 1]>1, :]
  

if __name__ == '__main__':
  print(query('stars.csv')) 



# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker
if __name__ == '__main__':
  # Compare your function output to the SQL query
  result = query('stars.csv')

In [ ]:
# Write your query function here

import numpy as np

def query(f_name):
  data = np.loadtxt(f_name, delimiter=',', usecols=(0, 2))
  filtered_data = data[data[:,1]>1, :]
  sorted_data = filtered_data[np.argsort(filtered_data[:, 1]), :]
  return sorted_data
 


# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker
if __name__ == '__main__':
  # Compare your function output to the SQL query
  result = query('stars.csv')

In [ ]:
# Write your query function here

import numpy as np

def query(fname_1, fname_2):
  stars = np.loadtxt(fname_1, delimiter=',', usecols=(0, 2))
  planets = np.loadtxt(fname_2, delimiter=',', usecols=(0, 5))

  f_stars = stars[stars[:,1]>1, :]                
  s_stars = f_stars[np.argsort(f_stars[:, 1]), :] 
 
  final = np.zeros((1, 1))
  for i in range(s_stars.shape[0]):
    kep_id = s_stars[i, 0]
    s_radius = s_stars[i, 1]

    matching_planets = planets[np.where(planets[:, 0] == kep_id), 1].T
    final = np.concatenate((final, matching_planets/s_radius))

  return np.sort(final[1:], axis = 0)
    


# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker
if __name__ == '__main__':
  # Compare your function output to the SQL query
  result = query('stars.csv', 'planets.csv')